# History on Demand - HDAT API

## Overview

The History on Demand (HoD) APIs provide access to global, historical weather data. Historical weather data is prevalent in trend analysis and the training of analytical models pertaining to energy, agriculture, insurance and many other industries. These APIs allows you to collect that data the way that works best for you. Each has its own style of interaction, allowable query types, delivery mode, and restrictions. Select the API that suits your usage style, desired data format and delivery mechanism.

The History on Demand (HoD) - HDAT API looks at the same data HoD Direct does. There are two variants. "HDAT r2" presents only the Gridded Currents on Demand (gCOD) data, and "HDAT Ext" additionally presents the Agriculture and Energy (AGE) data.
In both cases, the goal of the API is to provide daily summaries of the data, rather than the raw hourly data. By summaries, we mean the Min, Max, and Average values for 7 "dayparts" for each day. Note that only 11 of the 30+ gCOD layers are present. This is because the others are either accumulated values already or are enumerations upon which math makes no sense.

This Notebook will demonstrate the usage of History on Demand (HoD) - HDAT API.

## Setup

All of the documentation and samples we provide are built using Jupyter notebooks.

To run the notebooks, below are the basic requirements:

1. A Python3.7 or higher environment
2. A Jupyter Notebook environment
Note: The notebooks include runnable examples but you need to run them in order from top to bottom. This is because there are some lines of set up code, for example, setting authentication credentials, that only appear in the first code cell. We leave them out in the subsequent cells to avoid clutter.

To run this notebook seamlessly, you have to first configure your EI API credentials in a file named `secrets.ini` in the below format:

```
[EI]
api.api_key = <Your EI API key>
api.org_id = <Your EI Org Id>
api.tenant_id = <Your EI Tenant Id>
```

Keep the secrets.ini file at an appropriate relative location of this notebook. For example, as specified in the below config.

```config.read('../../auth/secrets.ini')```

Execute this notebook, cell after cell

## History on Demand - HDAT API Sample

Let us start with the initial imports required for calling the HOD HDAT API

In [1]:
import requests
from pprint import pprint
import pandas as pd
import configparser
from io import StringIO
import json
from pandas import json_normalize
from datetime import datetime
import matplotlib.pyplot as plt

Now, we will set up the API Credentials as listed in below code

In [2]:
config = configparser.RawConfigParser()
config.read('../../auth/secrets.ini')

# The below line is written so that the table's data won't get truncated.
pd.set_option('display.max_colwidth', None)

EI_API_KEY     = config.get('EI', 'api.api_key')
EI_ORG_ID      = config.get('EI', 'api.org_id') 
EI_TENANT_ID   = config.get('EI', 'api.tenant_id')

EI_AUTH_ENDPOINT = "https://api.ibm.com/saascore/run/authentication-retrieve"
EI_API_BASE_URL  = "https://api.ibm.com/geospatial/run/v3/wx"
EI_API_ENDPOINT  = f"{EI_API_BASE_URL}/observations/historical/analytical/ext"

EI_AUTH_CLIENT_ID  = 'saascore-' + EI_TENANT_ID
EI_CLIENT_ID = 'geospatial-' + EI_TENANT_ID

auth_request_headers: dict = {}
auth_request_headers["X-IBM-Client-Id"] = EI_AUTH_CLIENT_ID
auth_request_headers["X-API-Key"] = EI_API_KEY

verify = True

auth_url = f"{EI_AUTH_ENDPOINT}/api-key?orgId={EI_ORG_ID}"
              
response = requests.get(url = auth_url,
                        headers = auth_request_headers,
                        verify  = verify
                       )
if response.status_code == 200:
    jwt_token = response.text
    print("Authentication Success")
else:     
    print("Authentication Failed")
    print(response.text)

Authentication Success


Once the API credentials are setup, we will build the required query parameters and headers for making an HOD HDAT API call. 
With the query parameters and the headers setup, we will send an HTTP GET Request to the HOD HDAT API Endpoint and get back the response.
This API response is a JSON structure that will be displayed.

In [3]:
# Create the query parameter
# provide the geocode cordinates, startdatetime, enddatatime, format as json, units as per your requirements to get data
query_params: dict = {}
query_params["postalKey"] = "10022:US"
query_params["startDate"] = "20230607"
query_params["endDate"] = "20230609"
query_params["format"] = "csv"
query_params["units"] = "s"
query_params["language"] = "en-US"

# Create the query headers
query_request_headers: dict = {}
query_request_headers["x-ibm-client-id"] = EI_CLIENT_ID
query_request_headers["Authorization"] = "Bearer " + jwt_token

# Create the Request object as GET Request with host URL, query parameters and query headers
request = requests.Request('GET', EI_API_ENDPOINT, params=query_params, headers=query_request_headers)
 
# Create a Session object and send the request
session = requests.Session()
response = session.send(request.prepare())

if response.text != "":
    # Get csv  and convert to dataframe
    csv_data = StringIO(response.text)
    dataframe = pd.read_csv(csv_data)
    
    # display
    #pprint(response_json)
    print("\n\n")
    pd.set_option('display.max_colwidth', None)
    display(dataframe) 
else:
    print("Empty Response")

,date,geoType,geoTypeId,latitude,longitude,DewpointLocalAfternoonAvg,DewpointLocalAfternoonMax,DewpointLocalAfternoonMin,DewpointLocalDayAvg,DewpointLocalDayMax,...,WindSpeedLocalEveningMin,WindSpeedLocalMorningAvg,WindSpeedLocalMorningMax,WindSpeedLocalMorningMin,WindSpeedLocalNighttimeAvg,WindSpeedLocalNighttimeMax,WindSpeedLocalNighttimeMin,WindSpeedLocalOvernightAvg,WindSpeedLocalOvernightMax,WindSpeedLocalOvernightMin
0,20230607,postalKey,10022:US,40.76923,-73.959979,277.1,278.1,275.9,278.3,280.4,...,1.7,3.1,4.5,2.2,1.3,2.5,0.1,0.6,1.3,0.1
1,20230608,postalKey,10022:US,40.76923,-73.959979,280.2,281.1,279.2,279.9,281.4,...,0.4,1.8,3.1,0.7,1.7,3.0,0.4,2.4,3.0,2.0
2,20230609,postalKey,10022:US,40.76923,-73.959979,281.0,281.8,280.1,281.5,282.6,...,0.4,1.5,2.4,0.4,1.9,3.4,0.4,2.2,2.6,1.9


### Understanding the sample:

<b>Code:</b>
The code basically is using the History on Demand (HOD) HDAT API endpoint to query the aggregated weather data for specific dates.
After creating the necessary API Credentials, a HTTP Get Request is created with query parameters and headers. The headers are the credentials, while the query parameters are the postal code, start and end days for the weather data to be queried, format of output, units and language.

The response of the History on Demand (HOD) HDAT API endpoint query is the csv data, that is transformed into dataframe for easy data intepretation and display. 

For more details on History on Demand (HOD) HDAT API refer - [HOD HDAT API Developer Guide](https://developer.ibm.com/apis/catalog/envintelsuite--ibm-environmental-intelligence/HDAT+API+Developer+Guide)

